In [2]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imgstore
from utilities import *
import stim_handling as stims


blacklist = [
             '/media/recnodes/recnode_2mfish/coherencetestangular3m_64_dotbot_20181023_131202.stitched', #fuzzy. bad tracking?
             '/media/recnodes/recnode_2mfish/coherencetestangular3m_64_dotbot_20181113_165201.stitched', #no stimulus
             '/media/recnodes/recnode_2mfish/coherencetestangular3m_128_dotbot_20181004_161201.stitched', #no stimulus
             '/media/recnodes/recnode_2mfish/coherencetestangular3m_128_dotbot_20181004_163201.stitched', #no stimulus
             '/media/recnodes/recnode_2mfish/coherencetestangular3m_128_dotbot_20181009_133201.stitched', #no stimulus
             '/media/recnodes/recnode_2mfish/coherencetestangular3m_256_dotbot_20181011_101202.stitched', #fuzzy. bad tracking?
             '/media/recnodes/recnode_2mfish/coherencetestangular3m_256_dotbot_20181204_113201.stitched', #no stimulus
             '/media/recnodes/recnode_2mfish/coherencetestangular3m_256_dotbot_20181214_135201.stitched', #truncated file
             '/media/recnodes/recnode_2mfish/coherencetestangular3m_1024_dotbot_20181025_103201.stitched', #fuzzy. bad tracking?
             '/media/recnodes/recnode_2mfish/coherencetestangular3m_1024_dotbot_20190201_105202.stitched', #truncated file.
             '/media/recnodes/recnode_2mfish/coherencetestangular3m_64_dotbot_20190529_115201.stit_WRONG_ched' #WRONG
             

            ]

groupsizes = [4,8,16,32,64,128,256,512,1024]
coherences = [0.0,0.2,0.4,0.6,0.8,1.0]

allData = pd.read_pickle('/media/recnodes/Dan_storage/20200120_coherence_data_compiled_full.pickle')
allData['variance'] = allData['std_dRotation_cArea']**2

In [3]:
DATE = '20200813'

In [5]:
allData[0:5]

,Timestamp,speed,dir,coh,median_dRotation_cArea,median_dRotation_cMass,std_dRotation_cArea,std_dRotation_cMass,pdfPeak1,pdfPeak1_height,pdfPeak2,pdfPeak2_height,entropy_Ra,syncTime,trialID,date,groupsize,variance
422,1.567677e+09,0.0,0.0,0.2,-0.459204,-0.441271,0.704379,0.582977,0.75,0.653013,0.55,2.528786,1.386294,-1 days +23:59:30.003157,20190905_115201_0,20190905,4,0.496150
423,1.567677e+09,0.0,0.0,0.2,-0.684161,-0.541690,0.732928,0.626750,0.75,0.653013,0.55,2.528786,1.386294,-1 days +23:59:30.023126,20190905_115201_0,20190905,4,0.537184
424,1.567677e+09,0.0,0.0,0.2,-0.769605,-0.540882,0.766816,0.620467,0.75,0.653013,0.55,2.528786,1.386294,-1 days +23:59:30.043098,20190905_115201_0,20190905,4,0.588007
425,1.567677e+09,0.0,0.0,0.2,-0.790903,-0.406638,0.790665,0.558103,0.75,0.653013,0.55,2.528786,1.386294,-1 days +23:59:30.063146,20190905_115201_0,20190905,4,0.625151
426,1.567677e+09,0.0,0.0,0.2,-0.782700,-0.359547,0.786908,0.551481,0.75,0.653013,0.55,2.528786,1.386294,-1 days +23:59:30.083076,20190905_115201_0,20190905,4,0.619225


In [7]:
'20190905_115201_0' in allData['trialID'].values

True

In [8]:
def sync_by_stimStart(df, col='speed'):
    """
    pass a df with stim information
    returns the df with rotation values adjusted for direction so rotation of new stim is positive
    """
    df = df.sort_values('Time')
    df.reset_index(inplace=True)
    df = df[:-10] #drop end to avoid dealing with annoying NaN cases #lazy #FIXME
    
    df.loc[:,'stimStart'] = 0
    firstStim = df.loc[df['Timestamp'] < df['Timestamp'].median(), 'speed'].idxmax()
    df.loc[firstStim, 'stimStart'] = 1
    df.loc[:,'stimEnd'] = 0
    lastStim = df.loc[df['Timestamp'] > df['Timestamp'].median(), 'speed'].idxmin()
    df.loc[lastStim, 'stimEnd'] = 1
    return df

In [ ]:
import glob

for fn in glob.glob('/media/recnodes/recnode_2mfish/coherencetestangular3m*.stitched/track/perframe_stats.pickle'):
    if fn.split('/track/perframe_stats')[0] in blacklist:
        print "excluding", fn
        continue
    elif '20181026' in fn:
        continue
    #don't process if already there. look for the trialID
    if fn.split('_dotbot_')[-1].split('.stitched')[0] + '_0' in allData['trialID'].values:
        continue
    
    print(fn)
    try:
        pf = pd.read_pickle(fn)
        if not 'frame' in pf.columns:
            ret, pf = stims.sync_data(pf, stims.get_logfile(fn.rsplit('/',2)[0]), imgstore.new_for_filename(fn.rsplit('/',2)[0] + '/metadata.yaml'))

        pf['dir'] = pd.to_numeric(pf['dir'], errors='coerce')
        pf['coh'] = pf['coh'].fillna(method='pad').fillna(method='backfill')

        pf = sync_by_stimStart(pf)
        fileID = fn.split('/')[-3].split('.')[0].split('_',3)[-1]
        aligned = align_by_stim(pf, fileID)
        aligned['groupsize'] = groupsize
        allData = pd.concat([allData, aligned], axis=0)
    except Exception as e:
        import sys, traceback
        exc_type, exc_value, exc_traceback = sys.exc_info()
        traceback.print_exception(exc_type, exc_value, exc_traceback, limit=2, file=sys.stdout)
        #print traceback.print_exception()
        print e
        pass
allData.to_pickle('/media/recnodes/recnode_2mfish/' + DATE + '_coherence_data_compiled_full.pickle')